In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from random import *
from math import pi, cos, exp, floor
from scipy.signal import StateSpace, impulse2, lsim2

In [ ]:
var = np.array([i*20 for i in range(10)])
j2 = [i for i in var if i >= 60]
j2

In [5]:
class gene:
    def __init__(self,size_):
        for i in range(size_):
            sVar = 'cell' + str(i)
            self.__dict__[sVar] = pd.DataFrame()
#G = gene(5)
#G.__dict__

In [6]:
class MOLIwhiteBox:
    def __init__(self,method):
        self.C = np.array([0, 0, 0, 1])
        self.A = np.zeros([4,4])
        if method is None:
            method = 'trivial'
        self.filter = method
        
    def describe(self):
        print('This is a MOLI white Box alertness model')
    def structure(self):
        if self.filter == 'trivial':
            w = pi/12
            tau = 28.12
            self.A[1,3] = -w**2
            self.A[2,3] = -w**2/tau
            self.A[3,3] = -1/tau
    
    def fit(self,timeSamp,u,y):
        A_ = np.transpose(self.A)
        C_ = np.transpose(self.C)
        I_ = np.identity(4)
        sFilt = StateSpace(A_,C_,I_,[])
        regres = np.array([])
        
        for i in range(u.shape[1]):
            impT, impResp = impulse2(sFilt, 0, timeSamp.iloc[i,:])
            outT, outResp = lsim2(sFilt, u.iloc[i,:], timeSamp.iloc[i,:])
            
            comp = np.zeros([u.shape[0],4*u.shape[1]])
            comp[:,4*i:4*(i+1)] = impResp[:,:]
            
            aux = np.concatenate((outResp,comp))
            regres = np.concatenate((regres,aux),1)
        
        _y = y.reshape([u.shape[0]*u.shape[1],1])
        theta = np.linalg.tensorsolve(regres,_y)
        
        self.L = theta[:3]
        for i in range(u.shape[1]):
            init = 3*i
            final = 3*i + 4
            self.B[:,i] = theta[init:final]

In [22]:
class simAlert:
    def __init__(self,numDays):
        self.days = numDays
        self.time = pd.DataFrame()
        self.levAl = pd.DataFrame()
        self.levHom = pd.DataFrame()
        self.omega = pi/12.0
        self.tau = 28.33
        self.M = 2.52
        self.phi = -4.41
        self.DC = 2.40
        self.y_ = 14.3
        self.tau_e = 2.6247
        self.smpAl = gene(numDays)
    
    def generate(self,windowDecision,resolution):
        initAl = 15.3
        self.resolution = resolution
        self.windows = windowDecision
        for i in range(self.days):
            unTime = [24*i + 24*j/self.resolution for j in range(10000)]
            wTime = [j - 24*i for j in unTime if (j - 24*i) <= windowDecision[i]]
            sTime = [j - 24*i - windowDecision[i] for j in unTime 
                     if (j - 24*i) >= windowDecision[i]]
            
            circadian = [self.M*cos(self.omega*k + self.phi) for k in unTime]
            wHom = [(initAl - self.DC)*exp(-k/self.tau) + self.DC for k in wTime]
            sHom = [self.y_*(1-exp(-k/self.tau_e)) + wHom[-1]*exp(-k/self.tau_e) 
                    for k in sTime]
            Hom = np.concatenate((wHom,sHom))
            
            self.levHom.loc[:,i] = Hom
            self.levAl.loc[:,i] = circadian + Hom
            self.time.loc[:,i] = unTime
            initAl = Hom[-1]
    
    def randSample(self,seed,ppH):
        
        size_ = floor(max(self.windows)*ppH)
        
        #self.sAl = np.zeros([size_,self.days])
        #self.sT = np.zeros([size_,self.days])
        
        for i in range(self.days):
            step = floor(self.resolution/(24*ppH))
            final_ = self.windows[i]*step*ppH
            samples = range(0,final_,step)
            spH = floor(self.resolution/24.0)
            cruzeArr = [k - floor(spH/4) for k in range(0,floor(spH/2),1)]
            cruzeInc = sample(cruzeArr, len(samples))
            if cruzeInc[0] < 0:
                cruzeInc[0] = 0
            if cruzeInc[-1] > 0:
                cruzeInc[-1] = 0
            samples_inc = np.asarray(cruzeInc) + samples
            size_ = samples_inc.shape[0]
            
            data_frame = {'time': self.time.loc[samples_inc,i].values,
                          'levAl': self.levAl.loc[samples_inc,i].values}
            self.smpAl.__dict__['_cell_'+str(i)] = data_frame
            
            #self.sAl[0:size_,i] = self.levAl.loc[samples_inc,i].values
            #self.sT[0:size_,i] = self.time.loc[samples_inc,i].values
            self.ind = spH
            

In [25]:
alertData = simAlert(2)
alertData.generate([16,16], 10000)
alertData.randSample(15,2)
#print(alertData.smpAl)
alertData.smpAl._cell_1

{'time': array([24.1704, 24.6384, 25.1448, 25.6104, 25.788 , 26.268 , 27.2304,
        27.7272, 28.0464, 28.5408, 28.8264, 29.5848, 30.132 , 30.6672,
        30.9288, 31.3968, 31.8888, 32.4024, 32.9424, 33.456 , 34.092 ,
        34.6344, 35.1024, 35.3064, 35.9904, 36.4632, 36.8448, 37.6416,
        37.956 , 38.3328, 38.952 , 39.456 ]),
 'levAl': array([13.15606062, 12.68259085, 12.19316358, 11.76832489, 11.61340379,
        11.21632943, 10.52585657, 10.22999075, 10.06297261,  9.84087024,
         9.73308207,  9.5199056 ,  9.43097783,  9.39454959,  9.39411532,
         9.42018789,  9.48216976,  9.58078712,  9.71673475,  9.87129423,
        10.08827856, 10.28782066, 10.46486952, 10.54219637, 10.79593289,
        10.96084277, 11.08411426, 11.3028652 , 11.37158916, 11.43867827,
        11.508858  , 11.52572221])}

In [ ]:
print(alertData.sT)

In [ ]:
plt.plot(alertData.time,alertData.levAl)
plt.scatter(alertData.sT,alertData.sAl)